In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df_red = pd.read_csv("winequality-red.csv", sep=';',index_col=False)
df_red['type'] = 1
df_white = pd.read_csv("winequality-white.csv", sep=';',index_col=False)
df_white['type'] = 0
df_wine = pd.concat([df_red, df_white], ignore_index=True)
X = df_wine.drop("citric acid", axis=1)
y = df_wine["citric acid"]

In [5]:
df_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [6]:
#misturar o df para fazer df training e test aleatorios
idx = np.arange(len(df_wine))
np.random.seed(42)
np.random.shuffle(idx)
# 80% train, 20% test
split_point = int(0.8 * len(df_wine))

train_idx = idx[:split_point]
test_idx = idx[split_point:]

X_train = X.iloc[train_idx]
y_train = y.iloc[train_idx]

X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]


In [7]:
continuous_cols = [col for col in X_train.columns if col != 'type']
binary_cols = ['type']  # ou a coluna que for binária

# Normaliza só as contínuas
scaler = (X_train[continuous_cols].mean(), X_train[continuous_cols].std())

X_train_cont = (X_train[continuous_cols] - scaler[0]) / scaler[1]
X_test_cont  = (X_test[continuous_cols]  - scaler[0]) / scaler[1]

# Junta de volta com a coluna binária (sem mexer nela)
X_train_scaled = pd.concat([X_train_cont, X_train[binary_cols]], axis=1)
X_test_scaled  = pd.concat([X_test_cont,  X_test[binary_cols]],  axis=1)

In [8]:
X_train_scaled  = np.asarray(X_train_scaled , dtype=float)
y_train = np.asarray(y_train, dtype=float)

X_test_scaled = np.asarray(X_test_scaled, dtype=float)
y_test = np.asarray(y_test, dtype=float)

In [14]:
class NN(object):
    def __init__(self):
        self.input = 12
        self.output = 1
        self.hidden_units1 = 64
        # initialize the matrix of weitghs
        np.random.seed(42)
        #w1 between input and hidden layer
        self.w1 = np.random.randn(self.input, self.hidden_units1) / np.sqrt(self.input)
        self.b1 = np.zeros((1, self.hidden_units1))
        self.w2 = np.random.randn(self.hidden_units1, self.output) / np.sqrt(self.hidden_units1)
        self.b2 = np.zeros((1, self.output))
    def _forward_propagation(self, X):
        # Camada 1
        self.z1 = X @ self.w1 + self.b1
        self.a1 = self._Relu(self.z1)
        
        # Camada de saida
        self.z2 = self.a1 @ self.w2 + self.b2
        self.a2 = self.z2       
        
        return self.a2
    def _Relu(self, x):
        return np.maximum(x,0)

    def _loss(self, predict, y):
        return np.mean((y-predict)**2)
    def _backward_propagation(self, X, y):
        m = X.shape[0]
        # Garante que y tenha shape correto (m, 1)
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        # Saída
        delta3 = (self.a2 - y) / m
        self.dw2 = self.a1.T @ delta3
        self.db2 = np.sum(delta3, axis=0, keepdims=True)
        
        # Camada escondida 1
        delta2 = (delta3 @ self.w2.T) * self._derivative_Relu(self.z1)
        self.dw1 = X.T @ delta2
        self.db1 = np.sum(delta2, axis=0, keepdims=True)
        
    def _derivative_Relu(self, u):
        return (u > 0).astype(float)
    def _update(self, learning_rate=0.01):

        self.w1 -= learning_rate * self.dw1
        self.w2 -= learning_rate * self.dw2
        self.b2 -= learning_rate * self.db2
        self.b1 -= learning_rate * self.db1
    def train(self, X, y, interations = 2000):
    
        for i in range(interations):
            y_hat = self._forward_propagation(X)
            loss = self._loss(y_hat, y)
            self._backward_propagation(X, y)
            self._update()
            if i%100 == 0:
                print(f"Erro na {i} intereacao:{loss}")
    def predict(self,X):
        y_hat = self._forward_propagation(X)
        return y_hat



In [15]:
y_train = np.array(y_train).reshape(-1, 1)
modelo = NN()
modelo.train(X_train_scaled, y_train)


Erro na 0 intereacao:0.30248328333434743
Erro na 100 intereacao:0.05060709259645337
Erro na 200 intereacao:0.036601685228327206
Erro na 300 intereacao:0.030561402650768545
Erro na 400 intereacao:0.02698511613711814
Erro na 500 intereacao:0.024562886037907957
Erro na 600 intereacao:0.02280389163149232
Erro na 700 intereacao:0.021465204519243032
Erro na 800 intereacao:0.020408430663271845
Erro na 900 intereacao:0.019550986581653773
Erro na 1000 intereacao:0.018839624949549282
Erro na 1100 intereacao:0.01823944977914257
Erro na 1200 intereacao:0.017725946931672322
Erro na 1300 intereacao:0.01728140964529242
Erro na 1400 intereacao:0.01689303423237513
Erro na 1500 intereacao:0.016551254547066224
Erro na 1600 intereacao:0.016248683676448543
Erro na 1700 intereacao:0.015978759863783738
Erro na 1800 intereacao:0.01573675854697818
Erro na 1900 intereacao:0.015518533944446345


In [16]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)
y_pred = modelo.predict(X_test_scaled)
y_test = np.array(y_test).reshape(-1, 1)
rmse_value = rmse(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse_value)
print("R²:", r2)



RMSE: 0.12567263021228883
R²: 0.2872844528995211
